In [1]:
import os
import random

import numpy as np
import quaternion

import xml.etree.cElementTree as ET
import xml.dom.minidom as minidom

from deap import base
from deap import creator
from deap import gp
from deap import tools
from deap import algorithms

from dm_control import mujoco
from dm_control.suite import base as control_base
from dm_control.suite import common
from dm_control.utils import containers
from dm_control.utils import rewards
from dm_control.rl import control
from dm_control.rl.control import PhysicsError

import matplotlib.pyplot as plt
from IPython.display import clear_output
import collections
from skimage import io, transform

from deap import base, creator, gp, tools
import operator
from genome_synth import Genesis, prettify
from findtarget import FindTarget, Physics

In [2]:
gen = Genesis()
print(gen.mk_leaf())
pset = gp.PrimitiveSet("MAIN", 0)
pset.addPrimitive(gen.combine_trees, 2)
pset.addTerminal(gen.mk_leaf)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("expr_init", gp.genFull, pset=pset, min_=2, max_=4)

toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr_init)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def get_model_and_assets():
    curpath = os.getcwd()
    return common.read_model(curpath + '/dust.xml'), common.ASSETS

def capture(_savename, individuals):
    imnames = set()
    picidx = 0
    for individual in individuals:
        creation = gp.compile(individual, pset)
        tree = ET.fromstring(prettify(gen.mk_model(creation)))
        tree = ET.ElementTree(tree)
        tree.write('dust.xml')
        _DEFAULT_TIME_LIMIT = 10
        _CONTROL_TIMESTEP = .04

        genesis_physics = Physics.from_xml_string(*get_model_and_assets())
        genesis_physics.set_genesis(gen)
        genesis_task = FindTarget()
        genesis_env = control.Environment(genesis_physics, 
                                         genesis_task,
                                         control_timestep=_CONTROL_TIMESTEP,
                                         time_limit=_DEFAULT_TIME_LIMIT)
        action_spec = genesis_env.action_spec()
        observation_spec = genesis_env.observation_spec()

        time_step = genesis_env.reset()
        curtime = 0.0
        while(not time_step.last()):
            try:
                action = np.ones(action_spec.shape[0]) * np.sin(curtime)
                time_step = genesis_env.step(action)
                savename = "basedgod_{0:04}.jpg".format(picidx)
                picidx += 1
                imnames.add(savename)
                curtime += _CONTROL_TIMESTEP
                io.imsave(savename, genesis_env.physics.render(64, 64, camera_id='tracking_top'))
            except PhysicsError:
                print('except')
                break
    if os.path.isfile('basedgod.mp4'):
        os.remove('basedgod.mp4')
    !!ffmpeg -f image2 -pattern_type sequence -i "basedgod_%4d.jpg" -qscale:v 0 basedgod.mp4
    for name in imnames:
        os.remove(name)
    
def evalGen(individual):
    creation = gp.compile(individual, pset)
    #print(creation)
    tree = ET.fromstring(prettify(gen.mk_model(creation)))
    tree = ET.ElementTree(tree)
    tree.write('dust.xml')
    _DEFAULT_TIME_LIMIT = 10
    _CONTROL_TIMESTEP = .04
    display_stride = 1 / .04 // 24
    picidx = 0
    imnames = set()
    
    genesis_physics = Physics.from_xml_string(*get_model_and_assets())
    genesis_physics.set_genesis(gen)
    genesis_task = FindTarget()
    genesis_env = control.Environment(genesis_physics, 
                                     genesis_task,
                                     control_timestep=_CONTROL_TIMESTEP,
                                     time_limit=_DEFAULT_TIME_LIMIT)
    action_spec = genesis_env.action_spec()
    observation_spec = genesis_env.observation_spec()
    
    
    idx = 0
    
    maxreward = 0.0
    max_energy_penalty = 0.0
    
    time_step = genesis_env.reset()
    curtime = 0.0
    
    while(not time_step.last()):
        try:
            action = np.ones(action_spec.shape[0]) * np.sin(curtime)
            time_step = genesis_env.step(action)
            reward = time_step.reward
            if(maxreward < reward):
                maxreward = reward
            curtime += _CONTROL_TIMESTEP
            idx += 1
        except PhysicsError:
            print('except')
            maxreward -= 500
            break
    return (maxreward,)

toolbox.register("evaluate", evalGen)
toolbox.register("select", tools.selTournament, tournsize=7)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=2, max_=4)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)




{'name': '0', 'offset': array([0., 0., 0.]), 'rotation': array([ 40.95412444, 173.90277148,  24.44798704]), 'dimensions': [0.13333835464436875, 0.055386131581885927, 0.055386131581885927], 'joint_range': 50.547516643153415, 'children': [], 'priority': 0.5060259656178392}


In [3]:
def main():
    pop = toolbox.population(n=15)
    hof = tools.HallOfFame(5)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    
    algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 5, stats, halloffame=hof)
    return pop, hof, stats

if __name__ == "__main__":
    pop, hoff, _ = main()
    capture("basedgod", hoff)
    
    

1
12
21
48
14
48
5
24
4
24
5
24
4
24
1
12
1
12
1
12
11
48
12
48
2
12
15
48
1
12
gen	nevals	avg     	std       	min     	max     
0  	15    	0.124176	0.00102899	0.120741	0.124986
11
33
19
50
2
15
0
9
18
50
3
15
17
50
0
9
21
50
4
21
1  	10    	0.122189	0.00524695	0.105451	0.125   
12
50
1
9
7
24
26
50
14
50
2  	5     	0.117151	0.0261107 	0.0200706	0.125   
17
50
15
50
23
50
10
30
26
50
23
50
21
50
6
24
23
50
18
50
11
42
3  	11    	0.167629	0.210836  	0.0709724	0.953468
27
50
15
50
21
50
1
15
15
50
6
30
25
50
4  	7     	0.232622	0.282798  	0.107213 	0.953468
23
48
17
50
33
50
24
50
19
50
20
48
23
50
13
50
19
50
16
48
14
50
1
12
5  	12    	0.220145	0.28856   	0.0390525	0.953468
23
50
28
50
10
24
16
50
11
50
